In [1]:
library(dplyr)
library(ggplot2)
library(readr)
library(reshape2)


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [2]:
df <- read_csv('../Datasets/car_price_prediction.csv')

Rows: 19237 Columns: 18
-- Column specification --------------------------------------------------------
Delimiter: ","
chr (13): Levy, Manufacturer, Model, Category, Leather interior, Fuel type, ...
dbl  (5): ID, Price, Prod. year, Cylinders, Airbags

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
df

ID,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
45654403,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005 km,6,Automatic,4x4,04-May,Left wheel,Silver,12
44731507,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000 km,6,Tiptronic,4x4,04-May,Left wheel,Black,8
45774419,8467,-,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000 km,4,Variator,Front,04-May,Right-hand drive,Black,2
45769185,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966 km,4,Automatic,4x4,04-May,Left wheel,White,0
45809263,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901 km,4,Automatic,Front,04-May,Left wheel,Silver,4
45802912,39493,891,HYUNDAI,Santa FE,2016,Jeep,Yes,Diesel,2,160931 km,4,Automatic,Front,04-May,Left wheel,White,4
45656768,1803,761,TOYOTA,Prius,2010,Hatchback,Yes,Hybrid,1.8,258909 km,4,Automatic,Front,04-May,Left wheel,White,12
45816158,549,751,HYUNDAI,Sonata,2013,Sedan,Yes,Petrol,2.4,216118 km,4,Automatic,Front,04-May,Left wheel,Grey,12
45641395,1098,394,TOYOTA,Camry,2014,Sedan,Yes,Hybrid,2.5,398069 km,4,Automatic,Front,04-May,Left wheel,Black,12


In [4]:
df <- select(df, -c('ID', 'Doors'))

In [5]:
df = unique(df)

In [6]:
df <- df[df$Levy != '-', ]

In [7]:
# Split the string and convert to integer
df$Mileage <- as.integer(sub(" .*", "", df$Mileage))

In [8]:
cols <- c('Price', 'Mileage', 'Levy')

df[cols] <- sapply(df[cols], as.numeric)

for (col in cols) {
  quartiles <- quantile(df[[col]], c(0.1, 0.8))
  q1 <- quartiles[1]
  q3 <- quartiles[2]

  low_bound <- q1 - 1.5 * (q3 - q1)
  upp_bound <- q3 + 1.5 * (q3 - q1)

  df <- df[df[[col]] >= low_bound & df[[col]] <= upp_bound, ]
}

In [9]:
table(df$Price)


    1     3     6    30    31    47    63    70    78    94   110   120   125 
    1     7     3     1     5     5     1     1     5    14     3     2     8 
  150   157   160   188   200   204   220   240   251   282   314   330   392 
    2    35     1    43     1     1    24     1    31    28    77     1   106 
  470   549   627   706   784   862   941  1019  1098  1176  1254  1333  1411 
  144   116    98    72    46    45    52    39    21    24    19    27    17 
 1490  1550  1568  1646  1725  1803  1882  1960  2000  2038  2117  2195  2200 
   21     1    23    29    28    15    22    18     1    23    10    14     2 
 2274  2352  2430  2509  2587  2600  2666  2744  2822  2901  2979  3000  3058 
   11    15    18    16    17     1    20     4    10    20    13     1     9 
 3136  3293  3450  3500  3600  3607  3763  3920  4077  4180  4234  4265  4300 
   34    56    28     1     1    19    24    22    37     1    25     1     2 
 4328  4359  4391  4547  4632  4679  4701  4704  47

In [10]:
cols <- c("Manufacturer", "Model", "Prod. year", "Category", "Leather interior", "Fuel type", "Engine volume", "Gear box type",
        "Drive wheels", "Wheel", "Color")

for (column in cols) {
  df[[column]] <- as.numeric(factor(df[[column]]))
}

In [11]:
columns <- c("Mileage", "Levy", "Price")

df[columns] <- scale(df[columns])

In [12]:
write_csv(df, '../Cleaned Datasets R/prices_cleaned.csv')